1. Модульные тесты для узла обнаружения объектов на изображениях:

In [ ]:
import unittest
from unittest.mock import patch, Mock
from sensor_msgs.msg import Image
from detection_msgs.msg import BoundingBox, ObjectDetection
from object_detection_node import ObjectDetectionNode

class TestObjectDetectionNode(unittest.TestCase):
    @patch('object_detection_node.ObjectDetectionNode.load_pretrained_model')
    def test_image_callback(self, mock_load_model):
        node = ObjectDetectionNode()
        mock_model = Mock()
        mock_model.detect.return_value = [(10, 20, 50, 100, 0, 0.8), (30, 40, 60, 80, 1, 0.7)]
        mock_load_model.return_value = mock_model

        image_msg = Image()
        node.image_callback(image_msg)

        self.assertEqual(len(node.publisher.msg.bounding_boxes), 2)
        self.assertEqual(node.publisher.msg.bounding_boxes[0].x, 10)
        self.assertEqual(node.publisher.msg.bounding_boxes[0].y, 20)
        self.assertEqual(node.publisher.msg.bounding_boxes[0].width, 50)
        self.assertEqual(node.publisher.msg.bounding_boxes[0].height, 100)
        self.assertEqual(node.publisher.msg.bounding_boxes[0].class_id, 0)
        self.assertEqual(node.publisher.msg.bounding_boxes[0].score, 0.8)

if __name__ == '__main__':
    unittest.main()


2. Модульные тесты для узла обнаружения объектов на лидарных данных:

In [ ]:
import unittest
from unittest.mock import patch, Mock
from sensor_msgs.msg import PointCloud2
from detection_msgs.msg import BoundingBox, ObjectDetection
from lidar_object_detection_node import LidarObjectDetectionNode

class TestLidarObjectDetectionNode(unittest.TestCase):
    @patch('lidar_object_detection_node.LidarObjectDetectionNode.load_lidar_object_detection_model')
    @patch('lidar_object_detection_node.LidarObjectDetectionNode.convert_pointcloud2_to_open3d')
    def test_pointcloud_callback(self, mock_convert_pointcloud, mock_load_model):
        node = LidarObjectDetectionNode()
        mock_model = Mock()
        mock_model.detect.return_value = [(10, 20, 50, 100, 0, 0.8), (30, 40, 60, 80, 1, 0.7)]
        mock_load_model.return_value = mock_model
        mock_convert_pointcloud.return_value = Mock()

        pointcloud_msg = PointCloud2()
        node.pointcloud_callback(pointcloud_msg)

        self.assertEqual(len(node.publisher.msg.bounding_boxes), 2)
        self.assertEqual(node.publisher.msg.bounding_boxes[0].x, 10)
        self.assertEqual(node.publisher.msg.bounding_boxes[0].y, 20)
        self.assertEqual(node.publisher.msg.bounding_boxes[0].width, 50)
        self.assertEqual(node.publisher.msg.bounding_boxes[0].height, 100)
        self.assertEqual(node.publisher.msg.bounding_boxes[0].class_id, 0)
        self.assertEqual(node.publisher.msg.bounding_boxes[0].score, 0.8)

if __name__ == '__main__':
    unittest.main()


3. Модульные тесты для узла объединения датчиков:

In [ ]:
import unittest
from unittest.mock import patch, Mock
from detection_msgs.msg import BoundingBox, ObjectDetection, FusedObjectDetection
from sensor_fusion_node import SensorFusionNode

class TestSensorFusionNode(unittest.TestCase):
    def test_fuse_detections(self):
        node = SensorFusionNode()

        rgb_detections = ObjectDetection()
        rgb_detections.bounding_boxes.append(BoundingBox(x=10, y=20, width=50, height=100, class_id=0, score=0.8))
        rgb_detections.bounding_boxes.append(BoundingBox(x=30, y=40, width=60, height=80, class_id=1, score=0.7))

        lidar_detections = ObjectDetection()
        lidar_detections.bounding_boxes.append(BoundingBox(x=15, y=25, width=45, height=95, class_id=0, score=0.7))
        lidar_detections.bounding_boxes.append(BoundingBox(x=35, y=45, width=55, height=75, class_id=1, score=0.6))

        fused_detections = node.fuse_detections(rgb_detections, lidar_detections)

        self.assertEqual(len(fused_detections), 2)
        self.assertEqual(fused_detections[0][0], (12.5, 22.5, 47.5, 97.5))
        self.assertEqual(fused_detections[0][1], 0)
        self.assertAlmostEqual(fused_detections[0][2], 0.75)
        self.assertEqual(fused_detections[1][0], (32.5, 42.5, 57.5, 77.5))
        self.assertEqual(fused_detections[1][1], 1)
        self.assertAlmostEqual(fused_detections[1][2], 0.65)

if __name__ == '__main__':
    unittest.main()


1. Документация для узла обнаружения объектов на изображениях:
"""
ROS2 Node for Object Detection on Images

This node is responsible for detecting objects in camera images using a pre-trained deep learning model.

Subscribes to:
    - /camera/image_raw (sensor_msgs/Image)

Publishes to:
    - /object_detections (detection_msgs/ObjectDetection)

Attributes:
    model (cv2.dnn.Net): Pre-trained object detection model.
    bridge (CvBridge): Bridge for converting ROS Image messages to OpenCV images.

Methods:
    load_pretrained_model(): Loads the pre-trained object detection model.
    image_callback(msg: sensor_msgs.msg.Image): Callback function for processing camera images.
"""
2. Документация для узла обнаружения объектов на лидарных данных:
"""
ROS2 Node for Object Detection on LiDAR Data

This node is responsible for detecting objects in LiDAR point cloud data using a pre-trained deep learning model.

Subscribes to:
    - /lidar/point_cloud (sensor_msgs/PointCloud2)

Publishes to:
    - /lidar_object_detections (detection_msgs/ObjectDetection)

Attributes:
    model (open3d.ml.torch.models.PointNet2SSGClassification): Pre-trained object detection model for LiDAR data.

Methods:
    load_lidar_object_detection_model(): Loads the pre-trained object detection model for LiDAR data.
    pointcloud_callback(msg: sensor_msgs.msg.PointCloud2): Callback function for processing LiDAR point cloud data.
    convert_pointcloud2_to_open3d(msg: sensor_msgs.msg.PointCloud2): Converts ROS PointCloud2 message to Open3D PointCloud.
"""
3. Документация для узла объединения датчиков:
"""
ROS2 Node for Sensor Fusion

This node is responsible for fusing the object detection results from the camera and LiDAR sensors.

Subscribes to:
    - /object_detections (detection_msgs/ObjectDetection)
    - /lidar_object_detections (detection_msgs/ObjectDetection)

Publishes to:
    - /fused_object_detections (detection_msgs/FusedObjectDetection)

Attributes:
    rgb_detections (detection_msgs.msg.ObjectDetection): Object detections from the camera sensor.
    lidar_detections (detection_msgs.msg.ObjectDetection): Object detections from the LiDAR sensor.

Methods:
    rgb_callback(rgb_detections: detection_msgs.msg.ObjectDetection): Callback function for processing camera object detections.
    lidar_callback(lidar_detections: detection_msgs.msg.ObjectDetection): Callback function for processing LiDAR object detections.
    fuse_detections(rgb_detections: detection_msgs.msg.ObjectDetection, lidar_detections: detection_msgs.msg.ObjectDetection): Fuses the object detections from camera and LiDAR sensors.
    is_bbox_overlap(bbox1: BoundingBox, bbox2: BoundingBox): Checks if two bounding boxes overlap.
    get_fused_bbox(bbox1: BoundingBox, bbox2: BoundingBox): Computes the fused bounding box.
    get_fused_class(class1: int, class2: int): Combines the object classifications.
"""
